# Лабораторная работа 1. Алгоритмы поиска подстроки в строке. 

### Мельников Евгений 18ПМИ-1, Екатерина Малышева 18ПМИ-2

In [ ]:
import time
from google.colab import drive
import os
import pandas as pd

drive.mount('/drive')
os.listdir('/drive/MyDrive/string_benchmarks') 


Mounted at /drive


['bad_t_1.txt',
 'bad_w_1.txt',
 'bad_t_2.txt',
 'bad_w_2.txt',
 'bad_t_3.txt',
 'bad_w_3.txt',
 'bad_t_4.txt',
 'bad_w_4.txt',
 'good_t_1.txt',
 'good_w_1.txt',
 'good_w_2.txt',
 'good_t_2.txt',
 'good_t_3.txt',
 'good_w_3.txt',
 'good_t_4.txt',
 'good_w_4.txt']

## Наивный алгоритм


In [ ]:
def naive_algo(string, pattern):  
  pattern_inds = []
  operations_count = 0

  start = time.time()
  for left_ind in range(len(string) - len(pattern) + 1):
    match_count = 0
    for pattern_right_ind in range(len(pattern)):
      right_ind = left_ind + pattern_right_ind

      operations_count += 1
      
      if string[right_ind] == pattern[pattern_right_ind]:
        match_count += 1
      else:
        break
      if match_count == len(pattern):
        pattern_inds.append(left_ind)
      

    #if text[left_ind:right_ind] == word:
    #  pattern_inds.append(left_ind)
    #right_ind +=1

  #pretty_print(text, word, pattern_inds)
  total_time = time.time() - start
  return pattern_inds, total_time, operations_count 


## Алгоритм Рабина-Карпа 


In [ ]:
x = [] # list with x's of different powers, where the index of an element represents the power of x 
# each element of the list is x to the power of index 


def hash_func(string, prev_hash=None, prev_first_letter_code=None):
  #print("calculating hash_code for %s previous hash: %s previous first letter: % s" %(string, prev_hash, prev_first_letter_code))
  # haven't found a better solution than using global variables to store the following data
  # I could've used OOP, but I'm way too lazy for this 
  global x

  q = 7919 # just a random prime
  
  if len(x) == 0: # filling up the list of powers of x 
    x_1 = 5011  # x to the power of 1
    for i in range(len(string)):
      x.append(x_1**i)

  resulting_hash = 0

  if prev_hash is None: # if we haven't calculated the hash yet 
    for i in range(len(string)):
      ascii_code = ord(string[i]) # ascii code of a character
      resulting_hash += ascii_code*(x[len(string) - 1 - i])
      resulting_hash = resulting_hash % q
      
  else: # if we have the hash for the previous prefix
    #resulting_hash = (((prev_hash - prev_first_letter_code) / x[1]) + (x[len(string)-1]*ord(string[-1]))) % q
    resulting_hash = ((prev_hash - prev_first_letter_code*x[len(string)-1])*x[1] + ord(string[-1])) % q
  
  #print('hashcode for %s: %s' %(string, resulting_hash))
  return resulting_hash, ord(string[0])


def rabin_karp_algo(string, substring):
  substring_inds = []
  candidate_hash = None
  first_letter_ascii = None
  global x
  x = []


  operations_count = 0
  start = time.time()
  substring_hash = hash_func(substring)[0] # hash code of a pattern  

  for string_ind in range(len(string) - len(substring) + 1):
    
    match_count = 0  

    candidate_hash, first_letter_ascii = hash_func(
        string[string_ind:string_ind+len(substring)],
        prev_hash=candidate_hash,
        prev_first_letter_code=first_letter_ascii)
    
    operations_count += 1
    if candidate_hash == substring_hash:    
      for substring_ind in range(len(substring)):
        operations_count += 1
        #print(substring_ind + string_ind, substring[substring_ind])
        if string[substring_ind + string_ind] == substring[substring_ind]:
          match_count += 1
        else:
          #print("Collision at", string_ind)
          break
      if match_count == len(substring):
        #print("Found a string at", string_ind)
        substring_inds.append(string_ind)
      
      #substring_inds.append(string_ind)

  total_time = time.time() - start
  '''
  if len(substring_inds) != 0:
    print("The substring has been found at", substring_inds)
  else:
    print("The substring hasn\'t been found")
  '''
  return substring_inds, total_time, operations_count


## Алгоритм бойера-мура-хорспула

In [ ]:
def encode(substring):
  substring_codes = []
  for i in range(len(substring) + 1):
    substring_codes.append(0)
  substring_codes[len(substring)] = len(substring) # *
 
 
  letter_offsets = {}
  letter_offsets['*'] = len(substring) 
  letter_offsets[substring[-1]] = len(substring) # last character
  
  for char_ind in range(len(substring) - 2, -1, -1): # from the (n-1)-th character down to the 0-th
    if substring[char_ind] not in letter_offsets:
      letter_offsets[substring[char_ind]] = letter_offsets['*'] - char_ind - 1
    elif substring[char_ind] in letter_offsets:
      letter_offsets[substring[char_ind]] = min(letter_offsets[substring[char_ind]], letter_offsets['*'] - char_ind - 1)
 
  return letter_offsets
 
 
def bmh_algo(string, substring):
  operations_count = 0
  #letter_offsets = encode(substring)
  #substring_right_ind = letter_offsets['*'] - 1
 
  string_right_ind = 0  
  current_offset = 0
  match_counter = 0
 
  not_found = False
  found_letter = False
 
  substring_inds = []

  start_time = time.time()
  
  letter_offsets = encode(substring)
  substring_right_ind = letter_offsets['*'] - 1
  while 1: 
    string_right_ind = substring_right_ind + current_offset
    if string_right_ind >= len(string): 
      break
 
 
    found_letter = False
    match_counter = 0
 
   # print(string_right_ind)
    operations_count += 1
    if string[string_right_ind] != substring[substring_right_ind]: # when last characters are different
      for substring_left_ind in range(substring_right_ind - 1, -1, -1): # going from left to right
        operations_count += 1
        if substring[substring_left_ind] == string[string_right_ind]: # if a matching letter is found 
          current_offset += letter_offsets[substring[substring_left_ind]] # then we offset
          found_letter = True 
          break # break, since we are looking for the first matching letter
 
      if not found_letter: # if the letter hasn't been found in the substring, then we offset by the length of the substring
        current_offset += letter_offsets["*"]
 
 
    else: # when last characters match
 
      for substring_left_ind in range(substring_right_ind - 1, -1, -1): # checking the rest of the characters
        
        string_left_ind = substring_left_ind + current_offset
        operations_count += 1
        if string[string_left_ind] == substring[substring_left_ind]:
          match_counter += 1
          #print(string[string_left_ind], substring[substring_left_ind], match_counter)
          if match_counter == letter_offsets['*'] - 1:
            #print("The substring %s has been found at %s " % (substring, string_left_ind))
            substring_inds.append(string_left_ind)
            #pretty_print(string, substring, [string_left_ind])
            current_offset += letter_offsets['*']
 
        else:
          current_offset += letter_offsets[substring[substring_right_ind - 1]]
 
  total_time = time.time() - start_time
  '''
  if len(substring_inds) != 0:
    print("The substring has been found at", substring_inds)
  else:
    print("The substring hasn\'t been found")
  '''
  return substring_inds, total_time, operations_count

## Алгоритм Кнута-Мориса-Пратта


In [ ]:
def prefsuff(word):
  pi=[]
  for i in range(len(word)):
    pi.append(0)
  pi[0]=0
  i, j=1, 0 # индексы 
  while i!=len(pi):
    if word[i]==word[j]:# если символы совпадают, двинаем индексы одновременно 
      pi[i]=j+1
      i+=1
      j+=1
    elif j==0: # если символы не совпадают и предыдущие не равны, тогда обнуляем pi[i] 
      pi[i]=0
      i+=1
    else:
      j=pi[j-1]# если предыдущие равны, откатываем до префикса поменьше
  return pi

def KNP(text, word):
  i=0 #индекс для текста
  j=0 #индекс для слова
  start=time.time()
  enter_ind=[]
  count_operations=0
  wordCount=0
  lenText=len(text)
  lenWord=len(word)
  p=prefsuff(word)
  while i!=lenText:
    count_operations+=1
    #print(i, j)
    if text[i]==word[j]:# если символ текста совпадает с символом слова, то двигаем индексы дальше "параллельно"
      i+=1
      j+=1
      if j == lenWord:
        #print("The substring has been found at", i-lenWord)
        enter_ind.append(i-lenWord)
        wordCount+=1
        j=0
    else:# если символы не равны и предыдущие не совпадали, сравниваем дальше с началом слова
      if j==0:
        i+=1
      else: # если символы не равно, но предыдущие совпадали, тогда сравниваем текст с префиксом слова(если есть)
        j = p[j-1]
  if wordCount==0:
    print("The substring hasn\'t been found")
  end = time.time()-start
  return enter_ind, end, count_operations

  
  


Алгоритм Ахо-Корасика



In [ ]:
class BorNode:
  def __init__(self):
        self.go_to = {} # словарь вершин - детей
        self.out = [] # слово от корня до текущей вершины
        #self.terminal=False
        self.fake = None # фейковая ссылка
    

def make_machine(words):
    if type(words) is not list:
        words=[words]
#создаем начальный автомат без ссылок 
    root = BorNode() #создаем пустой корень
    for word in words:
        node = root
        for symbol in word:
            node = node.go_to.setdefault(symbol, BorNode()) 
        node.out.append(word)
#Образуем ссылки:
#  От всех первых символов проводим фейковую ссылку в пустой корень и дальше обходим в ширину
    queue = []
    for node in root.go_to.values():
        queue.append(node)
        node.fake = root

    while len(queue) > 0:
        node1 = queue.pop(0) #родитель
        for key, node2 in node1.go_to.items():# идем по детям 
            queue.append(node2)
            node_fake = node1.fake # фейковая ссылка родителя
            while node_fake is not None and key not in node_fake.go_to: #если из фейковой ссылки родителя нет ребра в вершину с символом ребенка
                node_fake = node_fake.fake # то двигаемся наверх по фейковой ссылке к прародителям, пока не найдем переход в вершину с таким же символом как у ребенка
            if node_fake:# если нашли, устанавливаем фейковую ссылку
              node2.fake = node_fake.go_to[key] 
            else: # если не нашли, устанавливаем ссылку на корень
              node2.fake = root
            node2.out += node2.fake.out
    return root

def Aho_Korasik(text, words):
    enter_ind=[]
    operation_count=0
    start=time.time()
    root= make_machine(words)
    node=root
    
    for i in range(len(text)):
        operation_count+=1
        while node is not None and text[i] not in node.go_to:# если у текущей вершины нет ребенка с символом из текста
            node = node.fake #тогда переходим по фейковой ссылке
        if node is None:
            node = root
            continue
        node = node.go_to[text[i]]
        for word in node.out:
            enter_ind.append( i - len(word) + 1)
    end = time.time() - start 
    return enter_ind, end, operation_count


In [ ]:

time_df, operations_df, enter_ind_df = benchmark("/drive/MyDrive/string_benchmarks/").report()


Opening up input files...
Finished
Checking naive_algo
Checking rabin_karp_algo
Checking bmh_algo
Checking KNP
Checking Aho_Korasik


## Время выполнения

In [ ]:
time_df

,good1,good2,good3,good4,bad1,bad2,bad3,bad4
naive_algo,0.000442,0.000587,0.002124,0.005754,0.000007,0.000247,0.023726,1.291739
rabin_karp_algo,0.001047,0.002405,0.017487,0.022605,0.000014,0.000121,0.002076,0.063344
bmh_algo,0.000172,0.000286,0.001201,0.002868,0.000009,0.000090,0.000788,0.004303
KNP,0.000185,0.000339,0.001162,0.003259,0.000005,0.000053,0.000490,0.002858
Aho_Korasik,0.000305,0.001425,0.002139,0.004358,0.000012,0.000071,0.000694,0.008471


## Количество операций сравнения

In [ ]:
operations_df

,good1,good2,good3,good4,bad1,bad2,bad3,bad4
naive_algo,714,1158,3554,10714,18,910,90100,4001000
rabin_karp_algo,695,1159,3440,10716,11,101,1001,5001
bmh_algo,714,1114,3170,11098,18,190,1900,9000
KNP,713,1158,3539,10714,18,190,1900,9000
Aho_Korasik,694,1158,3438,10714,10,100,1000,5000


## Индексы вхождений подстроки в строку

In [ ]:
enter_ind_df

,good1,good2,good3,good4,bad1,bad2,bad3,bad4
naive_algo,[599],[610],[1629],[9522],[8],[90],[900],[4000]
rabin_karp_algo,[599],[610],[1629],[9522],[8],[90],[900],[4000]
bmh_algo,[599],[610],[1629],[9522],[8],[90],[900],[4000]
KNP,[599],[610],[1629],[9522],[8],[90],[900],[4000]
Aho_Korasik,[599],[610],[1629],[9522],[8],[90],[900],[4000]
